We actually worked on this algorithm in the main section but didn't get very good results, so this time we're going to make a simple modification to see if it works better. The modifications are mainly in the simplification of the algorithm and the simplification of the preprocessing.

The platform used is anaconda's jupyter Notebook platform and the environment is self-configured.The detailed configuration information can be found in the final report.

In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [ ]:
mbti_dataset_line = tf.data.TextLineDataset("DataBase_MBTI.csv")

In [ ]:
for ex in mbti_dataset_line.take(5):
    print(ex)

tf.Tensor(b'type,posts,typescat,IE,NS,TF,JP', shape=(), dtype=string)
tf.Tensor(b'INFJ,hello  ptypetoken  eostokendot  sorri  hear  your distress eostokendot   onli natur   relationship    perfect   time  everi moment  exist eostokendot    figur  hard time  time  growth  eostokendot  eostokendot  eostokendot     ,8,1,0,0,0', shape=(), dtype=string)
tf.Tensor(b'INFJ,prozac wellbrutin  least thirti minut  move your leg   dont mean move them while sit  your same desk chair weed  moder mayb  edibl   healthier altern eostokendot  eostokendot  eostokendot     ,8,1,0,0,0', shape=(), dtype=string)
tf.Tensor(b'INFJ,basic come  with three item youv determin that each type  whichev type  want   would more than like  given each type cognit function  whatnot when left  eostokendot  eostokendot  eostokendot     ,8,1,0,0,0', shape=(), dtype=string)
tf.Tensor(b'INFJ, thing  moder eostokendot   sim  inde  video game   good   that eostokendot  note  good   that  somewhat subject  that    complet promot 

In [ ]:
def label(line):
  label =  tf.strings.substr([line],[-10],[1])
  if label[0]==',':
    label = tf.strings.substr([line],[-9],[1])
  else:
    label = tf.strings.substr([line],[-10],[2])
  labelnum=tf.strings.to_number(label,tf.int64)
  line= tf.strings.substr([line],[6],(tf.strings.length([line])-17))
  return line[0], labelnum[0]


In [ ]:

mbti_dataset_line = mbti_dataset_line.skip(1).map(lambda line: label(line))


In [ ]:
for ex in mbti_dataset_line.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'ello  ptypetoken  eostokendot  sorri  hear  your distress eostokendot   onli natur   relationship    perfect   time  everi moment  exist eostokendot    figur  hard time  time  growth  eostokendot  eostokendot  eostokendot    '>, <tf.Tensor: shape=(), dtype=int64, numpy=8>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'rozac wellbrutin  least thirti minut  move your leg   dont mean move them while sit  your same desk chair weed  moder mayb  edibl   healthier altern eostokendot  eostokendot  eostokendot    '>, <tf.Tensor: shape=(), dtype=int64, numpy=8>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'asic come  with three item youv determin that each type  whichev type  want   would more than like  given each type cognit function  whatnot when left  eostokendot  eostokendot  eostokendot    '>, <tf.Tensor: shape=(), dtype=int64, numpy=8>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'thing  moder eostokendot   sim  inde  video game   good   that eo

In [ ]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [ ]:
mbti_dataset_line = mbti_dataset_line.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in mbti_dataset_line:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

83501

In [ ]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [ ]:
example_text = next(iter(mbti_dataset_line))[0].numpy()
print(example_text)

b' love  give  gift  freedom  someon els eostokendot   allow    love   their authent self without tri  chang  correct them   most unselfish    represent eostokendot  eostokendot  eostokendot    '


In [ ]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[1720, 5494, 11623, 15192, 5633, 29152, 46299, 37417, 1720, 35224, 15957, 78792, 28173, 1099, 20789, 48355, 33563, 904, 35304, 23706, 46299, 46299, 46299]


In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = mbti_dataset_line.map(encode_map_fn)

Split the dataset into test and train batches

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(30,), dtype=int64, numpy=
 array([ 1720,  5494, 11623, 15192,  5633, 29152, 46299, 37417,  1720,
        35224, 15957, 78792, 28173,  1099, 20789, 48355, 33563,   904,
        35304, 23706, 46299, 46299, 46299,     0,     0,     0,     0,
            0,     0,     0])>, <tf.Tensor: shape=(), dtype=int64, numpy=9>)

In [ ]:
vocab_size += 1

In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(tf.keras.layers.Embedding(vocab_size+1, 64))

In [ ]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [ ]:
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

model.add(tf.keras.layers.Dense(16, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, epochs=1, validation_data=test_data)

4203/4203 [==============================] - 482s 115ms/step - loss: 2.2360 - accuracy: 0.2242 - val_loss: 2.2317 - val_accuracy: 0.2238


In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

     79/Unknown - 13s 168ms/step - loss: 2.2317 - accuracy: 0.2238
Eval loss: 2.232, Eval accuracy: 0.224


The results are still poor, and these two explorations prove that LSTM is indeed not suitable for this dataset and this problem.